# Projeto 1 - Ciência dos Dados

Nome: Guilermo Kuznietz

Nome: Jonas Bonfá Pelegrina

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/guigokuznietz/Desktop/P1-CDADOS2021.2


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'alexa_b.xlsx'

In [4]:
train = pd.read_excel(filename)
train.tail()

FileNotFoundError: [Errno 2] No such file or directory: 'alexa_b.xlsx'

In [ ]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

___
## Classificador automático de sentimento


<img src="img/alexa_logo.png" alt="Alexa logo" width="500" height="600">

Nosso projeto busca entender o uso da Alexa no mercado brasileiro como os usuário interagem com ela e quais são as principais reações sobre o produto.

Exemplos de casos relevantes:
- Alexa,como está o dia hoje?
- Eu e minha família adoramos a Alexa
- A Alexa me entende como vivia antes sem ela?

Exemplos de casos irrelevante:
- Queria comprar uma Alexa
- Eu amo minha amiga que chama Alexa
- Tenho quatro filhas: Beatriz,Maria e Alexa

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.max_rows = 13

In [ ]:
#Limpeza de mensagens removendo os caracteres especiais
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;@]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed
    

In [ ]:
cleanup('@duda_barroso1 alexa, o que tenho pra hj?')

In [ ]:
#RELEVANTE
train_relevante = train.loc[train['Classificacao'] == 1,'Treinamento']
lista = []
lista_relevante = []

for i in range(0,len(train_relevante)):
    lista.append(cleanup(train_relevante[i]).split())

for lista1 in lista:
    for palavra in lista1:
        if "https" not in palavra:
            lista_relevante.append(palavra)

#RELEVANTE
train_irrelevante = train.loc[train['Classificacao'] == 0,'Treinamento']
lista = []
lista_irrelevante = []

for frase in train_irrelevante:
    lista.append(cleanup(frase).split())

for lista1 in lista:
    for palavra in lista1:
        if "https" not in palavra:
            lista_irrelevante.append(palavra)

**Verificando as frequências**

In [ ]:
# RELEVANTE - FREQUENCIA ABSOLUTA
serie_relevante = pd.Series(lista_relevante)
serie_relevante.value_counts()

In [ ]:
# RELEVANTE - FREQUENCIA RELATIVA
serie_relevante = pd.Series(lista_relevante)
serie_relevante_relativa = serie_relevante.value_counts(True)

In [ ]:
# IRRELEVANTE - FREQUENCIA ABSOLUTA
serie_irrelevante = pd.Series(lista_irrelevante)
serie_irrelevante.value_counts()

In [ ]:
# IRRELEVANTE - FREQUENCIA RELATIVA
serie_irrelevante = pd.Series(lista_irrelevante)
serie_irrelevante_relativa = serie_irrelevante.value_counts(True)

In [ ]:
# TRANSFORMANDO EM UM GRANDE TEXTO

livro_relevante = " ".join(lista_relevante)

livro_irrelevante = " ".join(lista_irrelevante)

In [ ]:
# JUNTANDO TODAS AS PALAVRAS
todas_palavras = livro_relevante + livro_irrelevante
todas_palavras = todas_palavras.split()
series_todas_palavras = pd.Series(todas_palavras)
series_todas_palavras_relativas = series_todas_palavras.value_counts(True)
print(series_todas_palavras_relativas)

**PROBABILIDADES**

In [ ]:
serie_relevante_relativa['alexa']

In [ ]:
serie_irrelevante_relativa['alexa']

**P(R) E P(I)**

In [ ]:
probR = len(serie_relevante) / len(series_todas_palavras)
print(f"A probabilidade de ser relevante é {probR}")

In [ ]:
probI = len(serie_irrelevante) / len(series_todas_palavras)
print(f"A probabilidade de ser irrelevante é {probI}")

**Frequência Absoluta das palavras nos Tweets relevantes**

In [ ]:
livro_relevante_series = pd.Series(livro_relevante.split())
livro_relevante_series = livro_relevante_series.value_counts()
soma_relevantes = livro_relevante_series.sum()
print('Palavras Relevantes,totais:',soma_relevantes)
print('Palavras Relevantes,não repetidas: ',len(livro_relevante_series))

**Frequência Absoluta das palavras nos Tweets irrelevantes**

In [ ]:
livro_irrelevante_series = pd.Series(livro_irrelevante.split())
livro_irrelevante_series = livro_irrelevante_series.value_counts()
soma_irrelevantes = livro_irrelevante_series.sum()
print('Palavras Relevantes,totais:',soma_irrelevantes)
print('Palavras Relevantes,não repetidas: ',len(livro_irrelevante_series))

**FREQUÊNCIA ABSOLUTA TOTAL** <br>
Frequencia Absoluta com relevantes e irrelevantes juntas

In [ ]:
print("O total de palavras na base é de:",len(series_todas_palavras))

**Começando a calcular**

In [ ]:
test

In [ ]:
#LIMPANDO A BASE DE TESTE
for linha in range (0,len(test)):
    test.iloc[linha,0] = cleanup(test.iloc[linha,0])
    test.iloc[linha,0] = re.sub(r'http\S+', '', test.iloc[linha,0])
test.tail(100)

In [ ]:
#Limpeza OK,nessa caso,faremos daqui a pouco
#Suavização OK
frase = cleanup(test.iloc[20,0])
frase_quebrada = frase.split()
palavras_frase = pd.Series(frase_quebrada)
contador_palavras = len(palavras_frase)

In [ ]:
#SUAVIZAÇÃO DE LAPLACE SEGUINDO MODELO DO MONKEY LEARN
#RELEVANTE
frase_d_relevante = 1
for palavra in palavras_frase:
    if palavra in livro_relevante_series:
        frase_d_relevante = frase_d_relevante * (livro_relevante_series[palavra] + 1) / (soma_relevantes + len(todas_palavras))
    else:
        frase_d_relevante = frase_d_relevante * 1 / (soma_relevantes + len(todas_palavras))

In [ ]:
#SUAVIZAÇÃO DE LAPLACE SEGUINDO MODELO DO MONKEY LEARN
#IRRELEVANTE
# O for tem problema ser o mesmo?
frase_d_irrelevante = 1
for palavra in palavras_frase:
    if palavra in livro_irrelevante_series:
        frase_d_irrelevante = frase_d_irrelevante * (livro_irrelevante_series[palavra] + 1) / (soma_irrelevantes + len(todas_palavras))
    else:
        frase_d_irrelevante = frase_d_irrelevante * 1 / (soma_irrelevantes + len(todas_palavras))

In [ ]:
#Agora nós devemos classificar se esse é 0 ou 1?
p_relevante_d_frase = probR * frase_d_relevante
print(p_relevante_d_frase)

In [ ]:
p_irrelevante_d_frase = probI * frase_d_irrelevante
print(p_irrelevante_d_frase)

In [ ]:
p_relevante_d_frase > p_irrelevante_d_frase

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**